In [5]:
import json

with open('../Data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [6]:
documents[3]

{'intent': 'change_order',
 'question': 'hep changing some products of an order',
 'response': "Appreciate your message to us for assistance with changing the products in your order. We understand that it's important for you to have the right items. To help you with this, could you please provide the order number and specify which products you would like to change? This way, we can ensure that your order is exactly as you want it. We're here to support you throughout the process, so please feel free to ask any questions or share any additional details that may help us fulfill your request.",
 'category': 'ORDER',
 'id': '85fcd90d'}

In [3]:
prompt_template = """
You emulate a customer who's getting touch with the customer support.
Formulate 5 questions this customer might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete.
If possible, use as fewer words as possible from the record. 

The record:

intent: {intent}
question: {question}
response: {response}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [5]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    json_response = response.choices[0].message.content
    return json_response

In [6]:
from tqdm.auto import tqdm

In [7]:
results = {}

In [8]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions
    with open('my_dict.json', 'w') as json_file:
        json.dump(results, json_file)

  0%|          | 0/418 [00:00<?, ?it/s]

In [9]:
import pickle

In [10]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'results.bin'

In [3]:
results

{'04474359': '```json\n\n{\n\n  "questions": [\n\n    "I\'m trying to cancel order {{Order Number}} as stated in the record.",\n\n    "What should I do after logging into my online portal?",\n\n    "How can I find and select my specific \'{{Online Order Interaction}}\' section for my order with number {{Order Number}} on your site according to the instructions provided?",\n\n    "Are there additional steps or is it just navigating to cancel by ordering interaction as guided in case of difficulties regarding this process from customer support during their working hours ({{Customer Support Hours}}) via phone at {{Customer Support Phone Number}} or through Live Chat on your website ({{Website URL}})?",\n\n    "If I encounter any issues while following these steps with order number {{Order Number}}, can you assist me further?"\n\n  ]\n\n}\n\n```',
 'bacc4a72': '{\n   "questions": [\n      "Can I track my order by number with your help?",\n      "How do I verify account details to find out 

In [2]:
import json 
with open('my_dict.json', 'rt') as f_in:
    results = json.load(f_in)

In [13]:
import re

def clean_json_string(json_string):
    # Remove code block markers
    cleaned = re.sub(r'```json|```', '', json_string).strip()
    print(cleaned)
    # Use regex to find valid JSON portion (either array or object)
    match = re.search(r'(\[.*\]|\{.*\})', cleaned, re.DOTALL)
    return match.group(0) if match else cleaned  # Return matched JSON or the original string

parsed_results = {}

# Parsing each JSON string after cleaning
for doc_id, json_questions in results.items():
    print(doc_id)
    # print(json_questions)
    cleaned_json = clean_json_string(json_questions)
    print(cleaned_json)
    parsed_results[doc_id] = json.loads(cleaned_json)

04474359
{

  "questions": [

    "I'm trying to cancel order {{Order Number}} as stated in the record.",

    "What should I do after logging into my online portal?",

    "How can I find and select my specific '{{Online Order Interaction}}' section for my order with number {{Order Number}} on your site according to the instructions provided?",

    "Are there additional steps or is it just navigating to cancel by ordering interaction as guided in case of difficulties regarding this process from customer support during their working hours ({{Customer Support Hours}}) via phone at {{Customer Support Phone Number}} or through Live Chat on your website ({{Website URL}})?",

    "If I encounter any issues while following these steps with order number {{Order Number}}, can you assist me further?"

  ]

}
{

  "questions": [

    "I'm trying to cancel order {{Order Number}} as stated in the record.",

    "What should I do after logging into my online portal?",

    "How can I find and sele

JSONDecodeError: Expecting value: line 4 column 161 (char 254)

In [3]:
results

{'04474359': '```json\n\n{\n\n  "questions": [\n\n    "I\'m trying to cancel order {{Order Number}} as stated in the record.",\n\n    "What should I do after logging into my online portal?",\n\n    "How can I find and select my specific \'{{Online Order Interaction}}\' section for my order with number {{Order Number}} on your site according to the instructions provided?",\n\n    "Are there additional steps or is it just navigating to cancel by ordering interaction as guided in case of difficulties regarding this process from customer support during their working hours ({{Customer Support Hours}}) via phone at {{Customer Support Phone Number}} or through Live Chat on your website ({{Website URL}})?",\n\n    "If I encounter any issues while following these steps with order number {{Order Number}}, can you assist me further?"\n\n  ]\n\n}\n\n```',
 'bacc4a72': '{\n   "questions": [\n      "Can I track my order by number with your help?",\n      "How do I verify account details to find out 

In [4]:
import json
import re
import pandas as pd

def clean_json_string(json_string):
    # Remove code block markers
    cleaned = re.sub(r'```json|```', '', json_string).strip()
    
    # Use regex to find valid JSON portion (either array or object)
    match = re.search(r'(\[.*\]|\{.*\})', cleaned, re.DOTALL)
    return match.group(0) if match else cleaned  # Return matched JSON or the original string

parsed_results = {}

# Parsing each JSON string after cleaning
for doc_id, json_questions in results.items():
    try:
        cleaned_json = clean_json_string(json_questions)
        parsed_results[doc_id] = json.loads(cleaned_json)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON for doc_id {doc_id}: {e}")

Error parsing JSON for doc_id 00903b0f: Expecting value: line 4 column 161 (char 254)
Error parsing JSON for doc_id 8a8f2ffc: Expecting ',' delimiter: line 8 column 5 (char 331)
Error parsing JSON for doc_id de48fa29: Expecting value: line 5 column 5 (char 235)
Error parsing JSON for doc_id 7662432a: Invalid control character at: line 5 column 620 (char 758)
Error parsing JSON for doc_id 7e0f551d: Invalid control character at: line 5 column 210 (char 404)
Error parsing JSON for doc_id 26f69bce: Expecting property name enclosed in double quotes: line 7 column 5 (char 1328)
Error parsing JSON for doc_id c3251b6e: Invalid control character at: line 6 column 94 (char 288)
Error parsing JSON for doc_id fb805297: Expecting property name enclosed in double quotes: line 2 column 3 (char 4)
Error parsing JSON for doc_id af4392ff: Expecting ',' delimiter: line 2 column 1 (char 406)
Error parsing JSON for doc_id 11ce3376: Expecting ',' delimiter: line 5 column 67 (char 221)
Error parsing JSON for

In [7]:
doc_index = {d['id']: d for d in documents}

In [15]:
parsed_results.items()

dict_items([('04474359', {'questions': ["I'm trying to cancel order {{Order Number}} as stated in the record.", 'What should I do after logging into my online portal?', "How can I find and select my specific '{{Online Order Interaction}}' section for my order with number {{Order Number}} on your site according to the instructions provided?", 'Are there additional steps or is it just navigating to cancel by ordering interaction as guided in case of difficulties regarding this process from customer support during their working hours ({{Customer Support Hours}}) via phone at {{Customer Support Phone Number}} or through Live Chat on your website ({{Website URL}})?', 'If I encounter any issues while following these steps with order number {{Order Number}}, can you assist me further?']}), ('bacc4a72', {'questions': ['Can I track my order by number with your help?', 'How do I verify account details to find out about my specific order status via customer support?', "What is the process for ass

In [19]:
parsed_results

{'04474359': {'questions': ["I'm trying to cancel order {{Order Number}} as stated in the record.",
   'What should I do after logging into my online portal?',
   "How can I find and select my specific '{{Online Order Interaction}}' section for my order with number {{Order Number}} on your site according to the instructions provided?",
   'Are there additional steps or is it just navigating to cancel by ordering interaction as guided in case of difficulties regarding this process from customer support during their working hours ({{Customer Support Hours}}) via phone at {{Customer Support Phone Number}} or through Live Chat on your website ({{Website URL}})?',
   'If I encounter any issues while following these steps with order number {{Order Number}}, can you assist me further?']},
 'bacc4a72': {'questions': ['Can I track my order by number with your help?',
   'How do I verify account details to find out about my specific order status via customer support?',
   "What is the process fo

In [21]:
final_results = []

# for doc_id, questions in parsed_results.items():
#     category = doc_index[doc_id]['category']
#     for q in questions:
#         final_results.append((q, category, doc_id))

for doc_id, data in parsed_results.items():
    # Ensure the doc_id exists in the doc_index to retrieve the category
    if doc_id in doc_index:
        category = doc_index[doc_id]['category']
        
        # Check if `data` is already a list of questions or a dictionary containing 'questions'
        if isinstance(data, list):
            questions = data  # If it's already a list of questions
        elif isinstance(data, dict):
            questions = data.get('questions', [])  # If it's a dict, extract 'questions'
        
        for q in questions:
            final_results.append((q, category, doc_id))

In [22]:
final_results

[("I'm trying to cancel order {{Order Number}} as stated in the record.",
  'ORDER',
  '04474359'),
 ('What should I do after logging into my online portal?',
  'ORDER',
  '04474359'),
 ("How can I find and select my specific '{{Online Order Interaction}}' section for my order with number {{Order Number}} on your site according to the instructions provided?",
  'ORDER',
  '04474359'),
 ('Are there additional steps or is it just navigating to cancel by ordering interaction as guided in case of difficulties regarding this process from customer support during their working hours ({{Customer Support Hours}}) via phone at {{Customer Support Phone Number}} or through Live Chat on your website ({{Website URL}})?',
  'ORDER',
  '04474359'),
 ('If I encounter any issues while following these steps with order number {{Order Number}}, can you assist me further?',
  'ORDER',
  '04474359'),
 ('Can I track my order by number with your help?', 'ORDER', 'bacc4a72'),
 ('How do I verify account details 

In [9]:
import pandas as pd

In [23]:
df = pd.DataFrame(final_results, columns=['question', 'category', 'document'])

In [24]:
df

,question,category,document
0,I'm trying to cancel order {{Order Number}} as...,ORDER,04474359
1,What should I do after logging into my online ...,ORDER,04474359
2,How can I find and select my specific '{{Onlin...,ORDER,04474359
3,Are there additional steps or is it just navig...,ORDER,04474359
4,If I encounter any issues while following thes...,ORDER,04474359
...,...,...,...
1662,Where is my {{Order Number}}?,ORDER,39a2fdee
1663,Can I see where order number has gone in the s...,ORDER,39a2fdee
1664,How to find out current status for said {{Orde...,ORDER,39a2fdee
1665,I need help understanding real-time tracking. ...,ORDER,39a2fdee


In [25]:
df.to_csv('../Data/ground-truth-data.csv', index=False)